<a href="https://colab.research.google.com/github/rajeshbhavikatti/daily_blink/blob/main/BLink_Notion_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#full project

### Initial Install of python modules and packages

In [59]:
!pip install cloudscraper
!pip install rich
!pip install tomd
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Initial Imports of the libraries

In [60]:
import cloudscraper
from datetime import datetime
from pathlib import Path
from rich.console import Console
from rich.progress import track
import tenacity
import requests,json
import tomd

### SCraping Blinkist daily for free daily book

In [61]:
BASE_URL = 'https://www.blinkist.com/'

HEADERS = {
    'x-requested-with': 'XMLHttpRequest',
}

In [62]:
CLOUDFLARE_MAX_ATTEMPTS = 10
CLOUDFLARE_WAIT_TIME = 2

LOCALES = ['en']

###creating scraper using cloudscraper

In [63]:
console = Console()
scraper = cloudscraper.create_scraper()

In [64]:
'''
@tenacity.retry(
    retry=tenacity.retry_if_exception_type(cloudscraper.exceptions.CloudflareChallengeError),
    wait=tenacity.wait_fixed(CLOUDFLARE_WAIT_TIME),
    stop=tenacity.stop_after_attempt(CLOUDFLARE_MAX_ATTEMPTS),
    before_sleep=lambda retry_state: console.print(f"Retrying in {retry_state.next_action.sleep} seconds…"),
)
'''

'\n@tenacity.retry(\n    retry=tenacity.retry_if_exception_type(cloudscraper.exceptions.CloudflareChallengeError),\n    wait=tenacity.wait_fixed(CLOUDFLARE_WAIT_TIME),\n    stop=tenacity.stop_after_attempt(CLOUDFLARE_MAX_ATTEMPTS),\n    before_sleep=lambda retry_state: console.print(f"Retrying in {retry_state.next_action.sleep} seconds…"),\n)\n'

In [65]:
def _request(url, **kwargs):
    """
    Wrapper for verifying and retrying GET requests.
    """
    kwargs.setdefault('headers', HEADERS)
    response = scraper.get(url, **kwargs)
    #response = client.get(url, **kwargs)
    # handle Cloudflare errors
    # We don't check the reponse content here; it could be large binary data and slow things down.
    if response.status_code == 403:
        # TODO: reset scraper for the next try?
        raise cloudscraper.exceptions.CloudflareChallengeError()

    response.raise_for_status()  # handle other errors
    return response

In [66]:
def _api_request(endpoint, params=None):
    """
    Wrapper for verifying and retrying GET requests to the Blinkist API.
    Returns the parsed JSON response.
    Calls `_request` internally.
    """
    url = f"{BASE_URL}api/{endpoint}"
    response = _request(url, params=params, headers=HEADERS)
    return response.json()

In [67]:
def get_free_daily(locale):
    # see also: https://www.blinkist.com/en/content/daily
    return _api_request('free_daily', params={'locale': locale})


def get_chapters(book_slug):
    
    return _api_request(f'books/{book_slug}/chapters')['chapters']


def get_chapter(book_id, chapter_id):
    #print(_api_request(f'books/{book_id}/chapters/{chapter_id}'))
    return _api_request(f'books/{book_id}/chapters/{chapter_id}')

### Getting Details from the site into variables

In [68]:
daily_blink = get_free_daily('en')

### Details of the book

In [69]:
Book_Author = daily_blink['book']['author']
Book_Title = daily_blink['book']['title']
Book_Cover = daily_blink['book']['image']['default']['src']
Book_Info = f'{Book_Title} by {Book_Author}'
Sub_Title = daily_blink['book']['subtitle']
about_book = daily_blink['book']['aboutTheBook']
synopsis = tomd.convert(about_book.strip())

###Getting Chapter Details from Site #this is take from a github repo

In [70]:
for locale in LOCALES:
    with console.status(f"Retrieving free daily in {locale}…"):
        free_daily = get_free_daily(locale=locale)
    book = free_daily['book']
    console.print(f"Today's free daily in {locale} is: “{book['title']}”")

    # list of chapters without their content
    with console.status(f"Retrieving chapters of {book['title']}…"):
        chapter_list = get_chapters(book['slug'])

    # fetch chapter content
    chapters = [get_chapter(book['id'], chapter['id'])
                for chapter in track(chapter_list, description='Fetching chapters…')]

Output()

Today's free daily in en is: “Ultralearning”

Output()

Output()

### Converting the Scraped data into Dictionary 

In [71]:
free_daily={} #Creating a empty dictionary for the free daily chapter name and article
for i in range(len(chapters)):
  page_name = str("Chapter " + str(i+1))
  page_article = tomd.convert(chapters[i]['text'].strip())
  free_daily[page_name]=page_article
#created a free_daily dictionary with key as page_name (eg. Chapter 1) and value as page_article which is the Chapter for that page

##Notion Connection


##notion Body

In [72]:
token = ''  #notion api access token

databaseId = ''     #database id 



### New Notion Page Body Basic Details


In [73]:
newPageData = {
        "icon": {
                "type": "external",
                "external": {
                    "url": Book_Cover
                }
            },
        "parent":{"database_id": databaseId},
        #database_id is the id of the database where the new page is to be created
        #"parent": {"type":"database_id", "database_id": databaseId },
        "properties": {
            "Book Title": {
                    "title": [
                        {
                            "type": "text",
                            "text": {
                                "content": Book_Title,
                            },
                        }
                    ]
                },
            "Author": {
                    "type": "select",
                    "select": {
                        "name": Book_Author,
                    }
                },
            "Sub Title": {
                "rich_text": [
                    {
                        "text": {
                            "content": Sub_Title,
                        }
                    }
                ]           },
            "Synopsis": {
                "rich_text": [
                    {
                        "text": {
                            "content": synopsis,
                        }
                    }
                ]           },
            "Book Cover": {
                    "type": "url",
                    "url": Book_Cover,
                },
        },
        "children": [
                     {
			"object": "block",
			"type": "heading_1",
			"heading_1": {
				"rich_text": [{ "type": "text", "text": { "content": Book_Info  } }]
			}
		},
    {
			"object": "block",
			"type": "heading_2",
			"heading_2": {
				"rich_text": [{ "type": "text", "text": { "content": "Rajesh Bhavikatti" } }],
      }
		},
    {
  "type": "image",
  "image": {
    "type": "external",
    "external": {
        "url": Book_Cover
    }
  }
}
    ]
    }


###creating and Adding children blocks to main new notion page Body

Creating child block array to add it to main page 

In [78]:
  page_child_data=[]
  for article_name,article in free_daily.items():
    page_child_data.append({  "object": "block",
      "type": "heading_1",'has_children':True,
      "heading_1": {
          "rich_text": [{ "type": "text", "text": { "content": article_name } }], # heading with chapter name eg Chapter 1
          "children": [{
              "object": "block",
              "type": "paragraph",
              "paragraph": {
                  "rich_text": [{"type": "text",
                                 "text": {"content": ""} }] #Empty 1st block under heading 
                            }
                        }]
                    }}
        )
    for line in article.splitlines():
      index = int(article_name[-2:])-1
      page_child_data[index]['heading_1']['children'].append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [{
                    "type": "text",
                    "text": {"content": line}  #add Line as Child block to heading (eg. Chapter 1)
                    }]
                    }
          })
    

Adding child block to main page block

In [75]:
for i in range(len(page_child_data)):
  newPageData['children'].append(page_child_data[i]) 

## notion api connection details and requesting to create a new NOTION page in given database id 

In [76]:

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}
createUrl = 'https://api.notion.com/v1/pages'

data = json.dumps(newPageData)
res = requests.request("POST", createUrl, headers=headers, data=data)

print(res.status_code)  # Check Status 
print(res.text)   # CHeck response

200
{"object":"page","id":"e8bdf9bd-58ab-48e1-98cd-4ebaf26fe05b","created_time":"2022-10-05T18:54:00.000Z","last_edited_time":"2022-10-05T18:54:00.000Z","created_by":{"object":"user","id":"216577b3-7744-4dc4-8323-ad43e288e49d"},"last_edited_by":{"object":"user","id":"216577b3-7744-4dc4-8323-ad43e288e49d"},"cover":null,"icon":{"type":"external","external":{"url":"https://images.blinkist.io/images/books/5d6c31e76cee0700081a65df/1_1/470.jpg"}},"parent":{"type":"database_id","database_id":"c9cd4aaa-1ad9-412e-9233-64f108843be9"},"archived":false,"properties":{"Sub Title":{"id":"PLvp","type":"rich_text","rich_text":[{"type":"text","text":{"content":"Accelerate Your Career, Master Hard Tasks and Outsmart the Competition","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"Accelerate Your Career, Master Hard Tasks and Outsmart the Competition","href":null}]},"Date":{"id":"Wwz%5C","type":"created_time","cr